In [1]:
import pandas as pd
import pysam
import click
from Bio import SeqIO
import gzip

In [23]:
polya_file = '/public/home/mowp/test/fast5_api/polyacaller.tsv'
fastq_file = '/public/home/mowp/test/nanopore_cdna/trim_data/fhh.qcat.fastq.gz'
out_fastq_file = 'test.fastq.gz'

In [19]:
# load tail_length and read_type data
df = pd.read_csv(polya_file, sep='\t', index_col=0)
tail_length = df['tail_length'].to_dict()
polya_type = df['read_type'].map(lambda x: False if x=='polyT' else True).to_dict()

In [22]:
pwd

'/public/home/mowp/test/nanopore_cdna/notebook'

In [ ]:
with gzip.open(fastq_file, 'rt') as f, gzip.open(out_fastq_file, 'wt') as o:
    for read in SeqIO.parse(f, 'fastq'):
        if read.id in tail_length:
            if not polya_type[read.id]:
                read.seq = read.seq.reverse_complement()
            SeqIO.write(read, o, 'fastq')

In [ ]:
@click.command()
@click.option('-i', '--infile', required=True, help='Input fastq file.', type=click.Path(exists=True))
@click.option('-p', '--polya_file', required=True, help='Input polya_file file.', type=click.Path(exists=True))
@click.option('-o', '--outfile', required=True, help='Output adjust file.')
def main(infile, outfile, polya_file):
    # load tail_length and read_type data
    df = pd.read_csv(polya_file, sep='\t', index_col=0)
    polya_type = df['read_type'].map(lambda x: False if x=='polyT' else True).to_dict()
    
    with gzip.open(infile, 'rt') as f, gzip.open(outfile, 'wt') as o:
        for read in SeqIO.parse(f, 'fastq'):
            if read.id in tail_length:
                if not polya_type[read.id]:
                    read.seq = read.seq.reverse_complement()
                SeqIO.write(read, o, 'fastq')